In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
sns.__version__

from sklearn.model_selection import TimeSeriesSplit

import lightgbm as lgb

#simpleallmodelstry
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score
from sklearn.metrics import fbeta_score
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn import svm
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)
sns.__version__
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# The usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sklearn
from sklearn.model_selection import (train_test_split, RandomizedSearchCV, 
                                     RepeatedStratifiedKFold, cross_validate)

# Assemble pipeline(s)
from sklearn import set_config
from sklearn.pipeline import make_pipeline, Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import make_scorer
# Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from sklearn.model_selection import GridSearchCV

## for plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn 

## for statistical tests
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import norm, skew
from scipy import stats
from scipy.special import boxcox1p
from catboost import CatBoostClassifier

import xgboost as xgb

C:\Users\hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\hp\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
networks=pd.read_csv(r"C:\Users\hp\Desktop\network.csv")

In [4]:
inspections=pd.read_csv(r"C:\Users\hp\Desktop\inspections.csv")

In [5]:
df=pd.merge(networks, inspections, on='PipeId', how='inner')


In [6]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'], format='%Y-%m-%d')

In [7]:
df["InspectionYear"]=df["InspectionDate"].dt.year
df["InspectionMonth"]=df["InspectionDate"].dt.month

In [8]:
df=df.reset_index()
df=df.drop(columns=["index"])

In [9]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'], format='%Y-%m-%d')

In [10]:
df["Density"] = np.zeros(len(df))
df.loc[df["GasType"]=="Gas natural", "Density"] = 1.88
df.loc[df["GasType"]=="Gas propano", "Density"] = 0.716

In [11]:
df["Diameter"]=df["Diameter"]/1000

In [12]:
df["Cross_sectional_area"]=(3.14*((df["Diameter"]/2)**2))

In [13]:
df["gas_flow_rate"]=(np.pi/4)* (df["Diameter"]**2) *np.sqrt((2*df["Pressure"]*(10**5))/df["Density"])#m3/hour

In [14]:
df["Velocity"]=df["gas_flow_rate"]/df["Cross_sectional_area"]#m/h

In [15]:
df['prev_inspection_date'] = df.groupby('PipeId')['InspectionDate'].shift(1)

In [16]:
df['prev_inspection_date'] = df['prev_inspection_date'].fillna(df["InspectionDate"])

In [17]:
mask = df['Incidence'] == 1
df.loc[mask, 'totalhourssincelastinspection'] = (df['InspectionDate'] - df['prev_inspection_date']).dt.total_seconds() / 3600
df['totalhourssincelastinspection'].fillna(0, inplace=True)

In [36]:
df["totalhoursingeneral"]= (df['InspectionDate'] - df['prev_inspection_date']).dt.total_seconds() / 3600
df['totalhoursingeneral'].fillna(0, inplace=True)

In [18]:
df["TotalExpectedVolumeSinceLeakage"]=df["gas_flow_rate"]*(df["totalhourssincelastinspection"]/2)

In [19]:
df["LeakedVolume"]=df["TotalExpectedVolumeSinceLeakage"]*0.2

In [37]:
df["TotalVolume"]=df["totalhoursingeneral"]*df["gas_flow_rate"]

In [38]:
df["TotalVolume"].sum()

508874832195.5976

As of of September 2021, the average price of natural gas in Spain was around 0.05-0.06 € per kWh (including taxes and other charges), which is equivalent to approximately 0.77-0.92 € per m3.

The price of propane gas in Spain can vary depending on several factors, such as the supplier, the location, and the volume of gas purchased. As an approximate estimate, the price of propane gas in Spain could range from 0.80 € to 1.20 € per liter, which is equivalent to approximately 1.17 € to 1.75 € per m3.

# Loss in terms of volume leakage cost

In [20]:
df["Lossineuros"]=0
df.loc[df["GasType"]=="Gas natural", "Lossineuros"] = 1.88*df["LeakedVolume"]
df.loc[df["GasType"]=="Gas propano", "Lossineuros"] = 0.716*df["LeakedVolume"]

In [21]:
df["Lossineuros"].sum()

63909547.745435394

In [53]:
df["carbonfootprintemitted"]=0

df.loc[df["GasType"]=="Gas natural", "carbonfootprintemitted"] = (df["LeakedVolume"] * 0.72 * 0.75 * 28) /1000000
df.loc[df["GasType"]=="Gas propano", "carbonfootprintemitted"] = (df["LeakedVolume"] * 1.88 * 0.819 * 3) / 1000000


In [54]:
df["carbonfootprintemitted"].sum()

512.4339426516806

# Loss in terms of repairing costs

The cost of repairing a gas pipe leak typically increases with the size of the leak and the extent of the damage. A small leak may only require a simple repair, such as tightening a fitting or replacing a small section of the pipe, which can be relatively inexpensive. However, if the leak is more significant or has caused damage to the surrounding area, such as the foundation of a building, the repair costs can be much higher.

Calculate the volume of gas in the pipe using the diameter, length, and cross-sectional area:

Volume = Length x Tcross sectional area

Calculate the mass of gas in the pipe using the density and volume:

Mass = Density x Volume

Calculate the flow rate of gas in the pipe using the flow rate and density:

Flow rate = Gas flow rate x Density

Calculate the time it takes for 1% of the gas volume to leak out of the pipe:

Time = (0.01 x Volume) / Flow rate

Estimate the size of the leak by calculating the flow rate of gas through the leak:

Leak flow rate = Volume / Time

Note that this assumes that the pressure in the pipe remains constant despite the leak, which may not be accurate in reality. Additionally, the size of the leak may change over time as the pressure in the pipe decreases due to the leak.

To estimate the size of the leak, we can use the following additional step:

Estimate the cross-sectional area of the leak using the leak flow rate and the velocity of gas through the leak.

The cross-sectional area of the leak can be estimated using the following equation:

Leak area = Leak flow rate / (Velocity x Gas density)

where Velocity is the velocity of the gas as it exits the orifice, which can be estimated using the Bernoulli's equation and assuming that the leak can be modeled as an orifice (see my previous answer for more details). Gas density is the density of the gas inside the pipe.

Once we have estimated the cross-sectional area of the leak, we can use it to estimate the size and shape of the leak, assuming that the leak is approximately circular in shape. This can be done using the following equation:

Leak diameter = 2 x sqrt(Leak area / pi)

Note that this assumes that the leak is a simple circular orifice and may not accurately represent the shape of the actual leak. Additionally, the velocity of gas through the leak may change over time as the pressure in the pipe decreases due to the leak, so this estimation may only be valid for a short period of time.


In [22]:
df["VolumePipe"]=df["Cross_sectional_area"]*df["Length"]

In [23]:
df["MassPipe"]=df["Density"]*df["VolumePipe"]

In [24]:
df["Flow_rate"]=df["gas_flow_rate"]*df["Density"]

In [25]:
df["time2leak"]=0
df.loc[df["Incidence"]==1,"time2leak"]=(0.02*df["VolumePipe"])/df["Flow_rate"]

In [26]:
df["leakflowrate"]=0
df.loc[df["Incidence"]==1,"leakflowrate"]=df["VolumePipe"]/df["time2leak"]

In [27]:
df["leakarea"]=df["leakflowrate"]/(df["Velocity"]*df["Density"])

In [28]:
df["leakdiameter"]=2*np.sqrt(df["leakarea"]/np.pi)

In [29]:
df["Leakdiameterassumed"]=df["leakdiameter"]+((df["leakdiameter"]*0.001)*(df["totalhourssincelastinspection"]/2))

Broadly speaking let's assume that gaz pipe leakage size costs 100$. Let's etimaate the repairing costs

In [30]:
df["repairingcostassumed"]=df["Leakdiameterassumed"]*100

In [31]:
df["repairingcostassumed"].sum()

6458975.573239356

# Number of leaks per InspectionLength

In [39]:
df[df["Incidence"]==1]

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,InspectionMonth,Density,Cross_sectional_area,gas_flow_rate,Velocity,prev_inspection_date,totalhourssincelastinspection,TotalExpectedVolumeSinceLeakage,LeakedVolume,Lossineuros,VolumePipe,MassPipe,Flow_rate,time2leak,leakflowrate,leakarea,leakdiameter,Leakdiameterassumed,repairingcostassumed,totalhoursingeneral,TotalVolume
12371,35790261,Córdoba,Lucena,2016,PE,Gas natural,0.0320,87.826000,0.15,12,0,False,ZRV-00007259,2018,2018-02-23,9,3,1,2,1.88,0.000804,0.101595,126.386861,2018-02-23,0.0,0.000000,0.000000,0.000000,0.070598,0.132724,0.190998,0.007393,9.549913,0.040192,0.226217,0.226217,22.621681,0.0,0.000000
12725,3663205,Granada,Granada,2012,PE,Gas natural,0.1100,194.409000,0.40,8,0,False,ZRV-00003435,2013,2013-05-15,16,3,1,5,1.88,0.009498,1.960385,206.388880,2013-05-15,0.0,0.000000,0.000000,0.000000,1.846594,3.471597,3.685523,0.010021,184.276169,0.474925,0.777620,0.777620,77.762027,0.0,0.000000
15123,228066029,Granada,Granada,1998,PE,Gas natural,0.2000,171.113000,0.15,5,0,False,ZRV-00003435,2017,2017-05-08,24,3,1,5,1.88,0.031400,3.968547,126.386861,2015-05-06,17592.0,34907.343218,6981.468644,13125.161050,5.372948,10.101143,7.460869,0.014403,373.043459,1.570000,1.413855,13.850124,1385.012401,17592.0,69814.686435
16173,331339238,Granada,Granada,2008,PE,Gas natural,0.1100,126.227181,0.15,0,0,False,ZRV-00003435,2019,2019-05-16,24,3,1,5,1.88,0.009498,1.200486,126.386861,2017-05-29,17208.0,10328.978088,2065.795618,3883.695761,1.198969,2.254061,2.256913,0.010625,112.845646,0.474925,0.777620,7.468265,746.826511,17208.0,20657.956177
16345,340505207,Granada,Granada,1998,PE,Gas natural,0.2000,46.319000,0.15,1,0,False,ZRV-00003435,2012,2012-03-22,22,3,1,3,1.88,0.031400,3.968547,126.386861,2012-03-22,0.0,0.000000,0.000000,0.000000,1.454417,2.734303,7.460869,0.003899,373.043459,1.570000,1.413855,1.413855,141.385504,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6328084,297705771,Córdoba,Palma del Río,2006,AO,Gas natural,0.1524,5.651000,49.50,1,0,False,ZRV-00002948,2012,2012-02-20,23,3,1,2,1.88,0.018232,41.859929,2295.931345,2012-02-20,0.0,0.000000,0.000000,0.000000,0.103030,0.193697,78.696667,0.000026,3934.833332,0.911611,1.077358,1.077358,107.735754,0.0,0.000000
6330435,303360970,Tarragona,Vandellos i l'Hospitalet,2007,AO,Gas natural,0.1016,5.574000,16.00,0,0,False,ZRV-00003026,2011,2011-06-16,24,3,1,6,1.88,0.008103,10.577264,1305.317887,2011-06-16,0.0,0.000000,0.000000,0.000000,0.045167,0.084915,19.885257,0.000045,994.262856,0.405160,0.718238,0.718238,71.823836,0.0,0.000000
6338580,343987428,Huelva,Lepe,2009,AO,Gas natural,0.2032,26.398000,16.00,0,0,False,ZRV-00004643,2017,2017-02-23,24,3,1,2,1.88,0.032413,42.309058,1305.317887,2015-02-14,17760.0,375704.432558,75140.886512,141264.866642,0.855634,1.608592,79.541029,0.000215,3977.051426,1.620642,1.436477,14.192390,1419.239004,17760.0,751408.865116
6341541,342771415,Lleida,Olius,2009,AO,Gas natural,0.1524,15.048000,16.00,0,0,False,ZRV-00004211,2015,2015-02-23,23,3,1,2,1.88,0.018232,23.798845,1305.317887,2013-03-12,17112.0,203622.917544,40724.583509,76562.216997,0.274358,0.515794,44.741829,0.000123,2237.091427,0.911611,1.077358,10.295229,1029.522869,17112.0,407245.835089


We can see that the index of the last detected leakage is 6341545 and that the total number of leakages is 12328

In [40]:
totallengthinspected=df.iloc[0:6344762]["Length"].sum()

The ratio of the number of leaks detected per meters inspected is

In [41]:
totallengthinspected/12328

18796.25979224656

In [42]:
totallengthinspected

231720290.71881562

In [45]:
grouped=df.groupby(["InspectionYear"])

def sum_until_last_incidence(group):
    incidence_1_indices = group.index[group['Incidence'] == 1]
    if len(incidence_1_indices) == 0:
        return 0, 0
    else:
        last_incidence_index = incidence_1_indices[-1]
        length_sum = group.loc[:last_incidence_index, 'Length'].sum()
        return length_sum


def sum_until_last_incidence2(group):
    incidence_1_indices = group.index[group['Incidence'] == 1]
    if len(incidence_1_indices) == 0:
        return 0, 0
    else:
        last_incidence_index = incidence_1_indices[-1]
        incidence_sum = group.loc[:last_incidence_index, 'Incidence'].sum()
        return incidence_sum

# apply the function to each group and get the sum for each year and PipeId
result = grouped.apply(sum_until_last_incidence)

result2=grouped.apply(sum_until_last_incidence2)
# reset the index to get the result as a DataFrame
result = result.reset_index()
result.columns = ['Year', 'LengthSum']
result2 = result2.reset_index()
result2.columns = ['Year', 'IncidenceSum']
result["IncidenceSum"]=result2["IncidenceSum"]
result["length_incidence_ratio"]=(result["IncidenceSum"]/result["LengthSum"])*100

In [48]:
result

,Year,LengthSum,IncidenceSum,length_incidence_ratio
0,2010,2.725252e+06,11,0.000404
1,2011,1.877556e+07,1311,0.006982
2,2012,2.130728e+07,1336,0.006270
3,2013,2.080382e+07,1166,0.005605
4,2014,2.062700e+07,974,0.004722
5,2015,2.212954e+07,832,0.003760
6,2016,2.243703e+07,1138,0.005072
7,2017,2.535961e+07,1763,0.006952
8,2018,2.403718e+07,1241,0.005163
9,2019,2.573383e+07,1413,0.005491


In [56]:
(result["IncidenceSum"].sum()/result["LengthSum"].sum())*100

0.005388422196191286